In [1]:
import os
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
from sklearn import preprocessing
from sklearn.svm import SVR
from math import sqrt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier,XGBRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# 原始資料
'''
columns = ['Age', 'Workclass', 'fnlwgt', 'education'
    , 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'salary']
'''
columns = ['Age','Workclass','fnlgwt','Education'
    ,'EdNum','MaritalStatus','Occupation','Relationship','Race','Sex','CapitalGain','CapitalLoss',
           'HoursPerWeek','Country','Income']


df_train_set = pd.read_csv('datasets/adult.data', names=columns,na_values=' ?')# na_values=' ?'
df_test_set = pd.read_csv('datasets/adult.test', names=columns, skiprows=1,na_values=' ?')# na_values=' ?'
df_train_set.head(20)
#print(df_train_set.shape)
#print(df_test_set.shape)

,Age,Workclass,fnlgwt,Education,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
df_train_set.drop('fnlgwt', axis=1, inplace=True)
df_test_set.drop('fnlgwt', axis=1, inplace=True)
df_train_set = df_train_set.dropna()
df_test_set = df_test_set.dropna()
print(df_train_set.shape)
print(df_test_set.shape)

(30162, 14)
(15060, 14)


In [4]:
df_train_set.drop(["Country", "Education"], axis=1, inplace=True) 
df_test_set.drop(["Country", "Education"], axis=1, inplace=True)
df_train_set.head(20)

,Age,Workclass,EdNum,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,Income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,<=50K
5,37,Private,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,<=50K
6,49,Private,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,<=50K
7,52,Self-emp-not-inc,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,>50K
8,31,Private,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,>50K
9,42,Private,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,>50K


In [5]:
colnames = list(df_train_set.columns) 
colnames.remove('Age')
colnames.remove('EdNum')
colnames = ['AgeGroup', 'EduGroup']+colnames

labels = ["{0}-{1}".format(i, i+9) for i in range(0,100,10)]
df_train_set['AgeGroup'] = pd.cut(df_train_set.Age, range(0,101,10), right = False, labels = labels)
df_test_set['AgeGroup'] = pd.cut(df_test_set.Age, range(0,101,10), right = False, labels = labels) 

labels = ["{0}-{1}".format(i,i+4) for i in range(0,20,5)]    
df_train_set['EduGroup'] = pd.cut(df_train_set.EdNum, range(0,21,5), right = False, labels = labels)   
df_test_set['EduGroup'] = pd.cut(df_test_set.EdNum, range(0,21,5), right = False, labels = labels)

df_train_set = df_train_set[colnames] 
df_test_set = df_test_set[colnames]


In [6]:
from sklearn_pandas import DataFrameMapper
mapper = DataFrameMapper([('AgeGroup', LabelEncoder()),('EduGroup', LabelEncoder()),
                          ('Workclass', LabelEncoder()),('MaritalStatus', LabelEncoder()),
                          ('Occupation', LabelEncoder()),('Relationship', LabelEncoder()),
                          ('Race', LabelEncoder()),('Sex', LabelEncoder()),
                          ('Income', LabelEncoder())], df_out=True, default=None)

cols = list(df_train_set.columns)
cols.remove('Income')
cols = cols[:-3]+['Income']+cols[-3:]  

df_train = mapper.fit_transform(df_train_set.copy())
df_train.columns = cols
df_test = mapper.fit_transform(df_test_set.copy())
df_test.columns = cols

scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
train_data_minmax = scaler.fit_transform(df_train[cols])
test_data_minmax = scaler.transform(df_test[cols])

df_train = pd.DataFrame(train_data_minmax,columns =cols)
df_test = pd.DataFrame(test_data_minmax,columns =cols)

cols.remove('HoursPerWeek')

In [7]:
x_train, y_train = df_train[cols].values, df_train['HoursPerWeek'].values
x_test, y_test = df_test[cols].values, df_test['HoursPerWeek'].values

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2,random_state=42)
df_train.head(10)

,AgeGroup,EduGroup,Workclass,MaritalStatus,Occupation,Relationship,Race,Sex,Income,CapitalGain,CapitalLoss,HoursPerWeek
0,0.250,0.333333,0.833333,0.666667,0.000000,0.2,1.0,1.0,0.0,0.021740,0.0,0.397959
1,0.500,0.333333,0.666667,0.333333,0.230769,0.0,1.0,1.0,0.0,0.000000,0.0,0.122449
2,0.250,1.000000,0.333333,0.000000,0.384615,0.2,1.0,1.0,0.0,0.000000,0.0,0.397959
3,0.500,1.000000,0.333333,0.333333,0.384615,0.0,0.5,1.0,0.0,0.000000,0.0,0.397959
4,0.125,0.333333,0.333333,0.333333,0.692308,1.0,0.5,0.0,0.0,0.000000,0.0,0.397959
5,0.250,0.333333,0.333333,0.333333,0.230769,1.0,1.0,0.0,0.0,0.000000,0.0,0.397959
6,0.375,1.000000,0.333333,0.500000,0.538462,0.2,0.5,0.0,0.0,0.000000,0.0,0.153061
7,0.500,1.000000,0.666667,0.333333,0.230769,0.0,1.0,1.0,1.0,0.000000,0.0,0.448980
8,0.250,0.333333,0.333333,0.666667,0.692308,0.2,1.0,0.0,1.0,0.140841,0.0,0.500000
9,0.375,0.333333,0.333333,0.333333,0.230769,0.0,1.0,1.0,1.0,0.051781,0.0,0.397959


In [8]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv, scoring_fit='neg_mean_squared_error',#neg_mean_squared_error
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

model = XGBRegressor()
param_grid = {
    'n_estimators': [100,110,120,130],
    'learning_rate': [0.06,0.07,0.08],
    'subsample': [1],
    'reg_lambda': [0],
    'min_child_weight' : [1],
    'colsample_bytree': [1] 
}

model, pred = algorithm_pipeline(x_train, x_validation, y_train, y_validation, model, 
                                 param_grid, cv=3)

# Root Mean Squared Error
print(np.sqrt(-model.best_score_))
print(model.best_params_)

y_pred=model.predict(x_validation)
y_true=y_validation
print("RMSE : %.4g" % metrics.mean_squared_error(y_true, y_pred,squared=False))


Fitting 3 folds for each of 1 candidates, totalling 3 fits
0.10708518852434623
{'colsample_bytree': 1, 'learning_rate': 0.07, 'min_child_weight': 1, 'n_estimators': 110, 'reg_lambda': 0, 'subsample': 1}
RMSE : 0.104


In [64]:
#sklearn.metrics.SCORERS.keys()

In [49]:
clf = XGBRegressor(
        #樹的個數
        n_estimators=110,
        # 如同學習率
        learning_rate= 0.07, 
        # 構建樹的深度，越大越容易過擬合    
        #max_depth=6, 
        # 隨機取樣訓練樣本 訓練例項的子取樣比
        subsample=1, 
        # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子
        gamma=0, 
        # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。
        reg_lambda=0,     
        #最大增量步長，我們允許每個樹的權重估計。
        max_delta_step=0,
        # 生成樹時進行的列取樣 
        colsample_bytree=1, 
        # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
        # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
        #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
        min_child_weight=1, 
        #隨機種子
        seed=0 
        # L1 正則項引數
#        reg_alpha=0,
        #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重
        #scale_pos_weight=1,
        #多分類的問題 指定學習任務和相應的學習目標
        #objective= 'multi:softmax', 
        # 類別數，多分類與 multisoftmax 並用
        #num_class=10,  
        # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。
#        silent=0 ,
        # cpu 執行緒數 預設最大
#        nthread=4, 
        #eval_metric= 'auc'
)


In [50]:
def mape_loss_func(labels,preds):
    mask=labels!=0
    return np.fabs((preds[mask]-labels[mask])/labels[mask]).mean()

In [51]:
clf.fit(x_train,y_train,eval_metric ='rmse')
y_pred=clf.predict(x_validation)

print("RMSE : %.4g" % metrics.mean_squared_error(y_validation, y_pred,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_validation,y_pred))


RMSE : 0.104
MAPE : 0.2777


In [52]:
clf.fit(x_train,y_train,eval_metric ='rmse')
y_pred=clf.predict(x_test)

print("RMSE : %.4g" % metrics.mean_squared_error(y_test, y_pred,squared=False))
print("MAPE : %.4g" % mape_loss_func(y_test,y_pred))

RMSE : 0.1067
MAPE : 0.286


C:\Users\cs040\anaconda3\envs\LAB_work\lib\site-packages\xgboost\data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +
